In [ ]:
# -*- coding: utf-8 -*-
"""
math.ipynb
====================================
Basic script to create the mathematical model (no best practices)

@author:
     - j.rodriguez.villegas
"""

In [ ]:
# Coding the convex relaxed linear problem using Pyomo

#Import the libraries

import numpy as np
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import matplotlib.pyplot as plt

from pyomo.environ import *

In [ ]:
# Declare the path to read the data
url = 'C:/Users/j.rodriguez.villegas/Documents/optimization/01_data/configuration_file.xlsx'

# Read the data
df = pd.read_excel(url, sheet_name = 'Historical_data_2')
df.head(15)

In [ ]:
# Create a pivot table
pivot_table = df.pivot(index="Date", columns="Asset", values="ROI")

# Calculate the covariance between asset returns (ROI)
covariance_matrix = pivot_table.cov()

# Get the list of unique assets
assets = df["Asset"].unique()

# Calculate and print the covariances for all combinations of products
for i in range(len(assets)):
    for j in range(i, len(assets)):
        asset1 = assets[i]
        asset2 = assets[j]
        covariance_prod = covariance_matrix.loc[asset1, asset2]
        print(f"Covariance ({asset1}, {asset2}): {covariance_prod}")

In [ ]:
# Create a pivot table
pivot_table = df.pivot(index="Date", columns="Asset", values="ROI")

# Calculate the correlation between asset returns (ROI)
correlation_matrix = pivot_table.corr()

# Get the list of unique assets
assets = df["Asset"].unique()

# Calculate and print the covariances for all combinations of products
for i in range(len(assets)):
    for j in range(i, len(assets)):
        asset1 = assets[i]
        asset2 = assets[j]
        correlation_prod = correlation_matrix.loc[asset1, asset2]
        print(f"Correlation ({asset1}, {asset2}): {correlation_prod}")

In [ ]:
# Calculate the expected return for each asset
# Create a new DataFrame
assets_df = df.groupby("Asset")["ROI"].mean().reset_index()
assets_df.rename(columns={"ROI": "Mean (average) ROI"}, inplace=True)

assets_df.head(10)

In [ ]:
# Optimization model (Convex linear relaxation)

assets_data = assets_df

# Sets
model = pe.ConcreteModel('markowitz')
model.assets = pe.Set(initialize = assets_data['Asset'].drop_duplicates())

assets_data = assets_data.set_index('Asset')

In [ ]:
print("The number of elements in the set {model.assets} is: ", len(model.assets))

In [ ]:
# Parameters
model.return_level = pe.Param(initialize = 0.001)
model.risk_level = pe.Param(initialize = 0.0001)
model.expected_return = pe.Param(model.assets, initialize = assets_data['Mean (average) ROI'].to_dict())

covariance_dict = {}
for i in range(len(assets)):
    for j in range(i, len(assets)):
        asset1 = assets[i]
        asset2 = assets[j]
        covariance_prod = covariance_matrix.loc[asset1, asset2]
        
        if asset1 not in covariance_dict:
            covariance_dict[asset1] = {}
        covariance_dict[asset1][asset2] = covariance_prod

model.covariance = pe.Param(model.assets, model.assets, initialize=0.0, mutable=True)

# Assign the covariance values to the parameter (product1, product2)
for asset1, covariance_i in covariance_dict.items():
    for asset2, covariance_prod in covariance_i.items():
        model.covariance[asset1, asset2] = covariance_prod

In [ ]:
# Display the values of model.covariance parameter
for asset1 in model.assets:
    for asset2 in model.assets:
        covariance_param = model.covariance[asset1, asset2].value
        print(f"Covariance ({asset1}, {asset2}): {covariance_param:.8f}")

In [ ]:
# Display the values of the model.expected_return parameter
for asset in model.assets:
    expected_return = model.expected_return[asset]
    print(f"Expected return ({asset}): {expected_return:.8f}")

In [ ]:
# Decision variables
model.w = pe.Var(model.assets, domain = pe.NonNegativeReals, bounds = (0,1))
model.u = pe.Var(model.assets, model.assets, domain = pe.NonNegativeReals)

In [ ]:
# Objective functions

# Maximization 
def calculate_total_return(model):
    '''
    This function calculates the total expected return on investment.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Return
    ------------
    double
        Total expected return.
    '''
    total_return = sum(model.expected_return[i] * model.w[i] for i in model.assets)
    return total_return

# Minimization
def calculate_total_risk(model):
    '''
    This function calculates the total risk of the portfolio.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Return
    ------------
    double
        Total risk.
    '''
    total_risk = sum(model.u[i,j] * model.covariance[i,j] for i in model.assets for j in model.assets)
    return total_risk

In [ ]:
# Constraints
def c_1_weights(model):
    '''
    Ensures the total weight sums up to one.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    weight_left = sum(model.w[i] for i in model.assets)
    weight_right = 1
    weight = (weight_left == weight_right)
    return weight

def c_2_return(model):
    '''
    Ensures the desired level of return of the portfolio is accomplished

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    expected_return_left = sum(model.expected_return[i] * model.w[i] for i in model.assets)
    expected_return_right = model.return_level
    expected_return = (expected_return_left >= expected_return_right)
    return expected_return

def c_3_1_linearized_risk(model):
    '''
    Linearizes the original MPT quadratic constraint by introducing an auxiliary variable.

    Parameters
    ----------

    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    auxiliary_1_left = sum(model.u[i,j] * model.covariance[i,j] for i in model.assets for j in model.assets)
    auxiliary_1_right = model.risk_level
    auxiliary_1 = (auxiliary_1_left <= auxiliary_1_right)
    return auxiliary_1

def c_3_2_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    lowBound = 0
    auxiliary_2_left = model.u[i,j]
    auxiliary_2_right = lowBound * model.w[j] + model.w[i] * lowBound  -  lowBound * lowBound
    auxiliary_2 = (auxiliary_2_left >= auxiliary_2_right)
    return auxiliary_2

def c_3_3_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    upperBound = 1
    auxiliary_3_left = model.u[i,j]
    auxiliary_3_right = upperBound * model.w[j] + model.w[i] * upperBound  -  upperBound * upperBound
    auxiliary_3 = (auxiliary_3_left >= auxiliary_3_right)
    return auxiliary_3

def c_3_4_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    lowBound = 0
    upperBound = 1
    auxiliary_4_left = model.u[i,j]
    auxiliary_4_right = upperBound * model.w[j] + model.w[i] * lowBound  -  upperBound * lowBound
    auxiliary_4 = (auxiliary_4_left <= auxiliary_4_right)
    return auxiliary_4

def c_3_5_linearized_risk(model, i, j):
    '''
    Assigns bounds to the auxiliary variable for the linear convex relaxation (McCormick Envelopes).

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    lowBound = 0
    upperBound = 1
    auxiliary_5_left = model.u[i,j]
    auxiliary_5_right = model.w[i] * upperBound  + lowBound * model.w[j]  -  lowBound * upperBound
    auxiliary_5 = (auxiliary_5_left <= auxiliary_5_right)
    return auxiliary_5

def c_4_non_negative(model, i):
    '''
    Non negativity in the decision variable.

    Parameters
    ----------

    model : Pyomo ConcreteMode
        The optimization model.
    i : string
        The assets.
    j : string.
        The assets.

    Returns
    -------
    Constraint Expression
        Relational expression for the constraint.
    '''
    non_negative_left = model.w[i]
    non_negatiave_right = 0
    non_negative = (non_negative_left >= non_negatiave_right)
    return non_negative


In [ ]:
# Solve the optimization model

from pyomo.environ import SolverFactory, SolverStatus, TerminationCondition

model.c_1_weights = pe.Constraint(rule = c_1_weights)
model.c_2_return = pe.Constraint(rule = c_2_return)
model.c_3_1_linearized_risk = pe.Constraint(rule = c_3_1_linearized_risk)
model.c_3_2_linearized_risk = pe.Constraint(model.assets, model.assets, rule = c_3_2_linearized_risk)
model.c_3_3_linearized_risk = pe.Constraint(model.assets, model.assets, rule = c_3_3_linearized_risk)
model.c_3_4_linearized_risk = pe.Constraint(model.assets, model.assets, rule = c_3_4_linearized_risk)

model.obj_calculate_total_return = pe.Objective(sense = pe.maximize, rule = calculate_total_return)

solver = pe.SolverFactory('glpk')

# Set the time limit in seconds
time_limit = 600  # 600 seconds

# GLPK configurations
solver.options['tmlim'] = time_limit

# Solve the linear convex problem
result = solver.solve(model, tee = True, report_timing = True)

# Check the solver's termination condition
termination_condition = result.solver.termination_condition
solver_status = result.solver.status

# Print the results or handle them accordingly
if solver_status == SolverStatus.ok and termination_condition == TerminationCondition.optimal:
    print("Optimal solution found.")
    pe.value(model.obj_calculate_total_return)
elif solver_status == SolverStatus.ok and termination_condition in [TerminationCondition.maxTimeLimit]:
    print("Time limit reached. No optimal solution found within the specified time.")
else:
    print("No optimal solution found, model is infeasible or unbounded...")
    print("Solver terminated with condition:", solver_status)

In [ ]:
print("Expected return: ", pe.value(model.obj_calculate_total_return))

In [ ]:
model.w.pprint()